In [1]:
import ollama

prompt = "Explain the concept of machine learning in simple terms."
response = ollama.generate(model="qwen2:0.5B", prompt=prompt)
print(response['response'])

# Chat completion
messages = [
    {
        'role': 'user',
        'content': 'Hello, who are you?'
    },
]
response = ollama.chat(model="qwen2:0.5B", messages=messages)
print(response['message']['content'])

Machine learning is a field that uses algorithms and statistical models to enable computers to learn from data, making predictions or decisions without being explicitly programmed. The goal of machine learning is to create systems or machines that can perform tasks such as recognizing images or speech, playing games like chess or cards, predicting outcomes in sports games, or even performing medical diagnoses based on patient information. Machine learning algorithms analyze large datasets to identify patterns and make predictions about the data's behavior. This allows computers to automatically learn from data without being trained by humans.
I am a model for this platform. I am called AI360


In [10]:
ollama.list()

{'models': [{'name': 'mistral:latest',
   'model': 'mistral:latest',
   'modified_at': '2024-06-29T11:30:00.454546851+02:00',
   'size': 4113301090,
   'digest': '2ae6f6dd7a3dd734790bbbf58b8909a606e0e7e97e94b7604e0aa7ae4490e6d8',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '7.2B',
    'quantization_level': 'Q4_0'}},
  {'name': 'qwen2:0.5b',
   'model': 'qwen2:0.5b',
   'modified_at': '2024-06-24T22:18:00.477255519+02:00',
   'size': 352164041,
   'digest': '6f48b936a09f7743c7dd30e72fdb14cba296bc5861902e4d0c387e8fb5050b39',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'qwen2',
    'families': ['qwen2'],
    'parameter_size': '494.03M',
    'quantization_level': 'Q4_0'}}]}

In [4]:
functions = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA"
                },
                "unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"]
                }
            },
            "required": ["location"]
        }
    }
]

In [ ]:
import ollama

def test_format(model_name, prompt, format_name):
    print(f"\nTesting {format_name} format:")
    print("Prompt:")
    print(prompt)
    
    response = ollama.generate(model=model_name, prompt=prompt)
    
    print("\nModel Response:")
    print(response['response'])
    print("-" * 50)

model_name = "mistral"

# Basic format with speaker tokens
basic_format = """<human>: What is the capital of France?</s>
<assistant>: The capital of France is Paris.</s>
<human>: And what about Germany?</s>
<assistant>:"""

# Numbered turns format
numbered_format = """1 USER: What's the capital of France?
1 ASSISTANT: The capital of France is Paris.
2 USER: And what about Germany?
2 ASSISTANT:"""

# Alpaca-style format
alpaca_format = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Tell me the capital of France.

### Response:
The capital of France is Paris.

### Instruction:
Now tell me the capital of Germany.

### Response:"""

# RWKV-style format
rwkv_format = """User: What's the capital of France?
Bot: The capital of France is Paris.
User: And what about Germany?
Bot:"""

formats = [
    (basic_format, "Basic format with speaker tokens"),
    (numbered_format, "Numbered turns format"),
    (alpaca_format, "Alpaca-style format"),
    (rwkv_format, "RWKV-style format")
]

for prompt, format_name in formats:
    test_format(model_name, prompt, format_name)

In [27]:
import json
import re

def parse_function_calls(s):
    """
    Parse a function call from a string.
    Example: '[{"name": "create_record", "arguments": {"name": "John"}}]\n\n[{"name": "create_record", "arguments": {"name": "Jane"}}]\n\n[{"name": "create_record", "arguments": {"name": "Bob"}}]\n\n"
    """
    # Find the function call JSON strings
    calls = re.findall(r"\[.*?\]", s)
    res = []
    print(s)
    for call in calls:
        try:
            parsed = json.loads(call)
            for p in parsed:
                res.append(p)
        except json.JSONDecodeError:
            continue
    return res

In [29]:
prompt = """
[AVAILABLE_TOOLS] 
[{"type": "function", "function": {"name": "get_current_weather", "description": "Get the current weather", "parameters": {"type": "object", "properties": {"location": {"type": "string", "description": "The city and state, e.g. San Francisco, CA"}, "format": {"type": "string", "enum": ["celsius", "fahrenheit"], "description": "The temperature unit to use. Infer this from the users location."}}, "required": ["location", "format"]}}}][/AVAILABLE_TOOLS]
[INST] What is the weather like today in San Francisco [/INST]
"""

c = ollama.generate(model="mistral", prompt=prompt, raw=True)
parse_function_calls(c['response'])

[{"name": "get_current_weather", "arguments": {"location": "San Francisco, CA", "format": "celsius"}}]


[{'name': 'get_current_weather',
  'arguments': {'location': 'San Francisco, CA', 'format': 'celsius'}}]

In [23]:
parse_function_call('[{"name": "create_record", "arguments": {"name": "John"}}]\n\n[{"name": "create_record", "arguments": {"name": "Jane"}}]\n\n[{"name": "create_record", "arguments": {"name": "Bob"}}]\n\nThese calls will create records with the names John, Jane, and Bob respectively.')

[{'name': 'create_record', 'arguments': {'name': 'John'}},
 {'name': 'create_record', 'arguments': {'name': 'Jane'}},
 {'name': 'create_record', 'arguments': {'name': 'Bob'}}]

In [24]:
parse_function_call("""
                    [FUNCTION_CALL][{"name": "create_record", "arguments": {"name": "Bob"}}]
[FUNCTION_CALL][{"name": "create_record", "arguments": {"name": "Sally"}}]
[/FUNCTION_CALLS]""")

JSONDecodeError: Expecting value: line 1 column 2 (char 1)